<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/4_Aprendizaje_no_supervisado/2_Solucion_Taller_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Análisis de Patrones de Consumo Internacional con Apriori**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

-
-

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Apriori_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

Hoy, 16 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

**Caso de Estudio: Consultoría para el PNUD**

**Contexto:** Una firma multinacional de e-commerce, "Global Retail Inc.", te ha contratado como consultor de datos. La empresa opera en múltiples países y ha notado que sus ventas y la efectividad de sus campañas de marketing varían significativamente entre regiones. Su hipótesis es que los patrones de compra y las asociaciones de productos son diferentes en cada mercado.

**Tu Misión:** Analizar el historial de transacciones de la empresa para descubrir y comparar las reglas de asociación de productos para dos de sus mercados más grandes: el Reino Unido y Alemania. Tu objetivo final es entregar recomendaciones de negocio accionables (ej. estrategias de cross-selling, promociones personalizadas) basadas en los patrones de consumo que descubras en cada país.

**Dataset:** Online Retail Data Set del repositorio de UCI. Encuentra mayor información en: https://archive.ics.uci.edu/dataset/352/online+retail

## Ejercicio 1: Configuración Inicial, Carga y Exploración de Datos

1.1 Importa las librerías necesarias

In [ ]:
### TU CÓDIGO AQUÍ ###
import os
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuraciones de visualización
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.2f}'.format

1.2 Carga el dataset "retail_top30_products.csv" que se encuentra en el repositorio del curso, carpeta "datasets".

In [ ]:
### TU CÓDIGO AQUÍ ###
path = 'C:\\Users\\Ferney Reina\\Documents\\Python\\IA_Economia\\2025-segundo-semestre-ia-economia\\datasets'
os.chdir(path)

In [ ]:
df = pd.read_csv('retail_top30_products.csv')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,"17,850.00",United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,"13,047.00",United Kingdom
2,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,"13,047.00",United Kingdom
3,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,"12,583.00",France
4,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,"12,583.00",France
...,...,...,...,...,...,...,...,...
40525,581579,22993,SET OF 4 PANTRY JELLY MOULDS,12,2011-12-09 12:19:00,1.25,"17,581.00",United Kingdom
40526,581583,20725,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,"13,777.00",United Kingdom
40527,581585,84879,ASSORTED COLOUR BIRD ORNAMENT,16,2011-12-09 12:31:00,1.69,"15,804.00",United Kingdom
40528,581585,22727,ALARM CLOCK BAKELIKE RED,4,2011-12-09 12:31:00,3.75,"15,804.00",United Kingdom


In [ ]:
print("Dimensiones del DataFrame:")
print(df.shape)

Dimensiones del DataFrame:
(40530, 8)


In [ ]:
print("\nInformación general del DataFrame:")
df.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40530 entries, 0 to 40529
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    40530 non-null  object 
 1   StockCode    40530 non-null  object 
 2   Description  40530 non-null  object 
 3   Quantity     40530 non-null  int64  
 4   InvoiceDate  40530 non-null  object 
 5   UnitPrice    40530 non-null  float64
 6   CustomerID   33004 non-null  float64
 7   Country      40530 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 2.5+ MB


1.3 Revisa si hay valores nulos en alguna columna y cuántos son

In [ ]:
### TU CÓDIGO AQUÍ ###
df.isna().sum()

InvoiceNo         0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     7526
Country           0
dtype: int64

1.4 Genera las estadísticas descriptivas de las variables numéricas

In [ ]:
### TU CÓDIGO AQUÍ ###
df[["Quantity", "UnitPrice", "CustomerID"]].describe()

,Quantity,UnitPrice,CustomerID
count,"40,530.00","40,530.00","33,004.00"
mean,11.38,4.64,"15,225.76"
std,40.69,57.76,"1,729.80"
min,"-1,930.00",0.00,"12,347.00"
25%,2.00,1.65,"13,767.00"
50%,5.00,2.95,"15,080.00"
75%,10.00,4.95,"16,745.00"
max,"2,880.00","8,142.75","18,283.00"


1.5 Observando las salidas del ejercicio anterior, ¿qué problemas potenciales identificas en las columnas Description, CustomerID y Quantity? ¿Ves algo inusual en los precios (UnitPrice)?

- **CostumetID:** con respecto a esta columna evidenciamos que exiten valores nulos, un total de 7526. Lo cual puede afectar el análisis, ya que no se podrá identificar a todos los clientes de manera única.
- **Description:** en esta columna no se evidencian problemas, ya que no hay valores nulos y los datos son string.
- **Quantity:** en esta columna no se evidencian problemas, ya que no hay valores nulos y los datos son enteros. Pero si se evidencian valores negativos, lo cual puede indicar devoluciones o errores en el registro de datos. Al igual que al realizar un analisis descriptivo es importante tener en cuanta que son productos diferentes y por tanto el analisis de las cantidades puede variar y estar sesgado segun producto.
- **UnitPrice:** en esta columna no se evidencian problemas, ya que no hay valores nulos y los datos son float. Pero si se evidencian preciosiguales 0, lo cual puede indicar donaciones o fallos en la captura de datos. Al igual que al realizar un analisis descriptivo es importante tener en cuanta que son productos diferentes y por tanto el analisis de los precios puede variar y estar sesgado segun producto y pais de origen

## Ejercicio 2: Limpieza y Preprocesamiento de Datos

Los datos del mundo real rara vez son perfectos. Antes de cualquier análisis, debemos "sanear" nuestro dataset. Completa el código en cada paso según las instrucciones.

2.1 **Manejo de Valores Nulos**: Las transacciones sin un CustomerID no son útiles para nosotros, ya que no podemos agrupar las compras de un cliente específico.

In [ ]:
# TAREA: Elimina todas las filas donde 'CustomerID' es nulo.
### TU CÓDIGO AQUÍ ###
df = df.dropna(subset=['CustomerID'])
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [ ]:
# El tipo de dato de CustomerID debe ser entero
### TU CÓDIGO AQUÍ ###
df['CustomerID'] = df['CustomerID'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33004 entries, 0 to 40529
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    33004 non-null  object 
 1   StockCode    33004 non-null  object 
 2   Description  33004 non-null  object 
 3   Quantity     33004 non-null  int64  
 4   InvoiceDate  33004 non-null  object 
 5   UnitPrice    33004 non-null  float64
 6   CustomerID   33004 non-null  int64  
 7   Country      33004 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.3+ MB


2.2 **Limpieza de Descripciones de Productos** Las descripciones pueden tener espacios en blanco al inicio o al final que podrían hacer que un mismo producto se cuente como dos diferentes.

In [ ]:
# TAREA: Limpia la columna 'Description' eliminando espacios extra al inicio y al final.
### TU CÓDIGO AQUÍ ###
df['Description'] = df['Description'].str.strip()

2.3 **Filtrado de Transacciones Anómalas**: Las facturas (InvoiceNo) que empiezan con 'C' indican una cancelación. Estas no son compras reales y deben ser eliminadas. Del mismo modo, las cantidades (Quantity) negativas representan devoluciones.

In [ ]:
# TAREA: Elimina las filas que correspondan a cancelaciones.
### TU CÓDIGO AQUÍ ###
df_limpio = df[~df['InvoiceNo'].str.contains('C', na=False)]

In [ ]:
# TAREA: Elimina las filas con cantidades negativas.
### TU CÓDIGO AQUÍ ###
df_limpio = df[df['Quantity'] > 0]

In [ ]:
# NO MODIFICAR ESTA CELDA
assert df_limpio['Quantity'].min() > 0, "Error: Todavía hay cantidades negativas."
assert df_limpio['CustomerID'].isnull().sum() == 0, "Error: Aún hay valores nulos en CustomerID."
assert df_limpio[df_limpio['InvoiceNo'].str.contains('C')].empty, "Error: Todavía hay facturas de cancelación."
print("¡La limpieza básica ha sido exitosa!")

¡La limpieza básica ha sido exitosa!


## Ejercicio 3: Análisis Comparativo por País

Ahora que los datos están limpios, vamos a segmentarlos y a aplicar el algoritmo Apriori para encontrar los patrones de compra en el Reino Unido y Alemania.

**Preparación de la Cesta de Mercado (Función)**

La siguiente función toma un dataframe, lo agrupa por factura y descripción, y lo transforma en el formato de matriz binaria que necesita el algoritmo Apriori. Estudia esta función, no necesitas modificarla.

In [ ]:
# NO MODIFICAR ESTA CELDA
def preparar_cesta(dataframe, pais):
    """Filtra por país y prepara la matriz de transacciones."""

    # Filtrar por el país de interés
    df_pais = dataframe[dataframe['Country'] == pais]

    # Crear la cesta: agrupar productos por factura
    cesta = (df_pais.groupby(['InvoiceNo', 'Description'])['Quantity']
             .sum().unstack().reset_index().fillna(0)
             .set_index('InvoiceNo'))

    # Convertir todas las cantidades positivas a 1 y todo lo demás a 0
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    cesta_encoded = cesta.applymap(encode_units)
    return cesta_encoded

3.1 Análisis para el Reino Unido (United Kingdom)

In [ ]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones del Reino Unido.
### TU CÓDIGO AQUÍ ###
pais = 'United Kingdom'
df_cesta = preparar_cesta(df_limpio, pais)
df_cesta.head()

Description,ALARM CLOCK BAKELIKE RED,ASSORTED COLOUR BIRD ORNAMENT,GREEN REGENCY TEACUP AND SAUCER,HEART OF WICKER SMALL,JAM MAKING SET PRINTED,JAM MAKING SET WITH JARS,JUMBO BAG PINK POLKADOT,JUMBO BAG RED RETROSPOT,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SUKI,LUNCH BAG BLACK SKULL.,LUNCH BAG APPLE DESIGN,LUNCH BAG CARS BLUE,LUNCH BAG PINK POLKADOT,LUNCH BAG RED RETROSPOT,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,NATURAL SLATE HEART CHALKBOARD,PACK OF 72 RETROSPOT CAKE CASES,PAPER CHAIN KIT 50'S CHRISTMAS,PARTY BUNTING,POSTAGE,RECIPE BOX PANTRY YELLOW DESIGN,REGENCY CAKESTAND 3 TIER,ROSES REGENCY TEACUP AND SAUCER,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 4 PANTRY JELLY MOULDS,SPOTTY BUNTING,WHITE HANGING HEART T-LIGHT HOLDER,WOODEN PICTURE FRAME WHITE FINISH
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
536367,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
536373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [ ]:
# TAREA: Aplica el algoritmo apriori para encontrar itemsets con un soporte mínimo de 2%.
### TU CÓDIGO AQUÍ ###
frequent_itemsets = apriori(df_cesta, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
frequent_itemsets.head()

,support,itemsets
27,0.19,(WHITE HANGING HEART T-LIGHT HOLDER)
7,0.14,(JUMBO BAG RED RETROSPOT)
22,0.14,(REGENCY CAKESTAND 3 TIER)
1,0.13,(ASSORTED COLOUR BIRD ORNAMENT)
20,0.13,(PARTY BUNTING)


In [ ]:
# TAREA: Genera las reglas de asociación. Queremos reglas con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###
rules_UK = association_rules(frequent_itemsets, metric="lift", min_threshold=3)
rules_UK = rules_UK.sort_values(by='lift', ascending=False)
rules_UK.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
64,"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",(GREEN REGENCY TEACUP AND SAUCER),0.04,0.06,0.03,0.75,12.37,1.00,0.02,3.80,0.95,0.38,0.74,0.59
65,(GREEN REGENCY TEACUP AND SAUCER),"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",0.06,0.04,0.03,0.43,12.37,1.00,0.02,1.70,0.98,0.38,0.41,0.59
67,(ROSES REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",0.07,0.03,0.03,0.39,12.24,1.00,0.02,1.59,0.98,0.36,0.37,0.61
62,"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(ROSES REGENCY TEACUP AND SAUCER),0.03,0.07,0.03,0.83,12.24,1.00,0.02,5.35,0.95,0.36,0.81,0.61
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.07,0.06,0.05,0.70,11.54,1.00,0.04,3.15,0.98,0.58,0.68,0.74


In [ ]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###
rules_UK =rules_UK[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by=['lift', 'confidence'], ascending=[False, False])
rules_UK.head(10)

,antecedents,consequents,support,confidence,lift
64,"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",(GREEN REGENCY TEACUP AND SAUCER),0.03,0.75,12.37
65,(GREEN REGENCY TEACUP AND SAUCER),"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",0.03,0.43,12.37
62,"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(ROSES REGENCY TEACUP AND SAUCER),0.03,0.83,12.24
67,(ROSES REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",0.03,0.39,12.24
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.05,0.70,11.54
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.05,0.78,11.54
78,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",(LUNCH BAG PINK POLKADOT),0.02,0.62,7.37
83,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",0.02,0.29,7.37
76,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",0.03,0.30,7.29
73,"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",(LUNCH BAG PINK POLKADOT),0.03,0.62,7.29


3.3 Observa las 3 reglas con el Lift más alto para el Reino Unido (1, 3 y 5). **Interprétalas:** ¿Qué te dicen estas asociaciones? ¿Qué tipo de productos son?

-En promedio de este analisis de las filas 1, 3 y 5 podemos evidenciar un lift en promedio superior a 11,54 lo cual estos nos dice que son productos complementarios y que se suelen vender juntos. Estos productos son relacionados al sector de la reposteria, dandonos claras señales de que los clientes que compran productos para reposteria tienden a comprar varios productos relacionados en una misma transacción. Esto puede indicar que los clientes están interesados en preparar postres o productos horneados, y por lo tanto, la empresa podría considerar estrategias de marketing dirigidas a este segmento específico, como promociones o descuentos en conjuntos de productos de repostería.

3.4 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

La tabla muestra reglas de asociación con soportes bajos (3–7%), lo que indica que los productos involucrados no son los más comprados, pero presentan relaciones muy fuertes. Las confianzas (0.39–0.83) evidencian que entre un 39 % y un 83 % de los clientes que adquieren el antecedente también compran el consecuente, reflejando una alta probabilidad de compra conjunta. El lift, superior a 11 en todas las reglas, confirma una fuerte complementariedad económica: la compra del antecedente hace que la del consecuente sea más de 11 veces más probable que al azar. En conjunto, estos indicadores sugieren oportunidades claras de venta cruzada entre artículos de la línea “Regency”

3.5 **Recomendación de Negocio:** Basado en estas reglas, ¿qué promoción o estrategia de venta específica podrías sugerir para el mercado del Reino Unido?

Se recomienda implementar una promoción combinada en el mercado del Reino Unido, ofreciendo descuentos o sets especiales que incluyan tazas y soportes para pasteles de la línea “Regency”. Dado el alto lift y la fuerte asociación entre estos productos, una estrategia de venta cruzada —por ejemplo, “compra un set de tazas y obtén descuento en el cakestand”— podría incrementar significativamente el ticket promedio y reforzar la fidelidad de clientes que buscan artículos de té y repostería de estilo clásico.

3.6 Análisis para Alemania (Germany)

In [ ]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones de Alemania.
### TU CÓDIGO AQUÍ ###
pais = 'Germany'
df_cesta_GER = preparar_cesta(df_limpio, pais)
df_cesta_GER.head()

Description,ALARM CLOCK BAKELIKE RED,ASSORTED COLOUR BIRD ORNAMENT,GREEN REGENCY TEACUP AND SAUCER,HEART OF WICKER SMALL,JAM MAKING SET PRINTED,JAM MAKING SET WITH JARS,JUMBO BAG PINK POLKADOT,JUMBO BAG RED RETROSPOT,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SUKI,LUNCH BAG BLACK SKULL.,LUNCH BAG APPLE DESIGN,LUNCH BAG CARS BLUE,LUNCH BAG PINK POLKADOT,LUNCH BAG RED RETROSPOT,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,NATURAL SLATE HEART CHALKBOARD,PACK OF 72 RETROSPOT CAKE CASES,PAPER CHAIN KIT 50'S CHRISTMAS,PARTY BUNTING,POSTAGE,RECIPE BOX PANTRY YELLOW DESIGN,REGENCY CAKESTAND 3 TIER,ROSES REGENCY TEACUP AND SAUCER,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 4 PANTRY JELLY MOULDS,SPOTTY BUNTING,WHITE HANGING HEART T-LIGHT HOLDER,WOODEN PICTURE FRAME WHITE FINISH
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536840,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
# TAREA: Aplica el algoritmo apriori con un soporte mínimo del 2%.
### TU CÓDIGO AQUÍ ###
frequent_itemsets_GER = apriori(df_cesta_GER, min_support=0.02, use_colnames=True)
frequent_itemsets_GER.sort_values(by='support', ascending=False, inplace=True)
frequent_itemsets_GER.head()

,support,itemsets
15,0.90,(POSTAGE)
17,0.15,(REGENCY CAKESTAND 3 TIER)
38,0.13,"(REGENCY CAKESTAND 3 TIER, POSTAGE)"
2,0.10,(JAM MAKING SET PRINTED)
14,0.09,(PACK OF 72 RETROSPOT CAKE CASES)


In [ ]:
# TAREA: Genera las reglas de asociación con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###
rules_GER = association_rules(frequent_itemsets_GER, metric="lift", min_threshold=3)
rules_GER = rules_GER.sort_values(by='lift', ascending=False)
rules_GER


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3,"(POSTAGE, JUMBO BAG RED RETROSPOT)",(JUMBO BAG PINK POLKADOT),0.07,0.04,0.03,0.45,11.63,1.00,0.03,1.74,0.98,0.41,0.43,0.63
4,(JUMBO BAG PINK POLKADOT),"(POSTAGE, JUMBO BAG RED RETROSPOT)",0.04,0.07,0.03,0.81,11.63,1.00,0.03,4.96,0.95,0.41,0.80,0.63
5,(JUMBO BAG RED RETROSPOT),"(JUMBO BAG PINK POLKADOT, POSTAGE)",0.09,0.03,0.03,0.36,11.53,1.00,0.03,1.52,1.00,0.36,0.34,0.68
2,"(JUMBO BAG PINK POLKADOT, POSTAGE)",(JUMBO BAG RED RETROSPOT),0.03,0.09,0.03,1.00,11.53,1.00,0.03,inf,0.94,0.36,1.00,0.68
0,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.04,0.09,0.04,0.94,10.81,1.00,0.03,14.61,0.94,0.41,0.93,0.68
1,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.09,0.04,0.04,0.42,10.81,1.00,0.03,1.65,0.99,0.41,0.39,0.68
6,(JAM MAKING SET PRINTED),(JAM MAKING SET WITH JARS),0.10,0.05,0.03,0.29,5.93,1.00,0.02,1.33,0.92,0.24,0.25,0.44
7,(JAM MAKING SET WITH JARS),(JAM MAKING SET PRINTED),0.05,0.10,0.03,0.60,5.93,1.00,0.02,2.25,0.87,0.24,0.55,0.44
9,"(POSTAGE, JAM MAKING SET WITH JARS)",(JAM MAKING SET PRINTED),0.04,0.10,0.02,0.56,5.56,1.00,0.02,2.05,0.85,0.18,0.51,0.39
10,(JAM MAKING SET PRINTED),"(POSTAGE, JAM MAKING SET WITH JARS)",0.10,0.04,0.02,0.21,5.56,1.00,0.02,1.22,0.91,0.18,0.18,0.39


In [ ]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###
rules_GER =rules_GER[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by=['lift', 'confidence'], ascending=[False, False])
rules_GER.head(10)

,antecedents,consequents,support,confidence,lift
4,(JUMBO BAG PINK POLKADOT),"(POSTAGE, JUMBO BAG RED RETROSPOT)",0.03,0.81,11.63
3,"(POSTAGE, JUMBO BAG RED RETROSPOT)",(JUMBO BAG PINK POLKADOT),0.03,0.45,11.63
2,"(JUMBO BAG PINK POLKADOT, POSTAGE)",(JUMBO BAG RED RETROSPOT),0.03,1.00,11.53
5,(JUMBO BAG RED RETROSPOT),"(JUMBO BAG PINK POLKADOT, POSTAGE)",0.03,0.36,11.53
0,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.04,0.94,10.81
1,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.04,0.42,10.81
7,(JAM MAKING SET WITH JARS),(JAM MAKING SET PRINTED),0.03,0.60,5.93
6,(JAM MAKING SET PRINTED),(JAM MAKING SET WITH JARS),0.03,0.29,5.93
9,"(POSTAGE, JAM MAKING SET WITH JARS)",(JAM MAKING SET PRINTED),0.02,0.56,5.56
10,(JAM MAKING SET PRINTED),"(POSTAGE, JAM MAKING SET WITH JARS)",0.02,0.21,5.56


3.7 Observa las 3 reglas con el Lift más alto para Alemania (1, 3 y 5). **Interprétalas:** ¿Qué patrones de consumo específicos del mercado alemán revelan estas reglas? ¿Son diferentes a los del Reino Unido?



En el mercado alemán, las reglas con mayor lift muestran una fuerte asociación entre las bolsas decorativas JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROSPOT y el servicio de POSTAGE. Esto indica que los clientes suelen comprar varias bolsas de diferentes diseños junto con servicios de envío, lo que sugiere un patrón de consumo orientado a regalos o empaques decorativos, más funcional y vinculado a la presentación que al uso personal.

A diferencia del mercado del Reino Unido, donde predominan asociaciones entre productos de mesa y decoración (tazas y soportes para pasteles), el patrón alemán refleja una preferencia práctica y de conveniencia, centrada en artículos para envolver y enviar productos, más que en la experiencia doméstica o estética.

3.8 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

En las reglas con mayor lift para Alemania, los soportes del antecedente y del consecuente (entre 3 % y 9 %) muestran que estos productos no son los más frecuentes en las transacciones, pero sí aparecen de forma consistente en conjunto. La confianza (0.36–0.81) indica que entre un 36 % y un 81 % de los clientes que compran uno de los productos (por ejemplo, una bolsa decorativa) también adquieren el otro o el servicio de envío, reflejando una alta probabilidad de compra conjunta. Finalmente, el lift (≈11.5) evidencia una fuerte complementariedad económica: la compra de una bolsa o el uso del servicio de envío hace que la probabilidad de adquirir el otro producto sea más de once veces superior a la esperada por azar. En conjunto, estos indicadores revelan un patrón de consumo sólido y coherente centrado en productos de regalo y empaques decorativos.

3.9 **Recomendación de Negocio:** ¿Qué campaña de marketing (diferente a la del Reino Unido) podrías diseñar para los clientes alemanes?

Para el mercado alemán, se recomienda una campaña enfocada en regalos y empaques personalizados, destacando la practicidad y el diseño de las bolsas decorativas. Una estrategia efectiva sería ofrecer promociones combinadas de “set de bolsas + envío gratuito” o descuentos por la compra de varios diseños, bajo un concepto como “Empaca con estilo: regala fácil, envía rápido”. Esta campaña respondería al patrón alemán de consumo funcional y orientado a la conveniencia, diferenciándose del enfoque británico centrado en la experiencia del hogar y la presentación de mesa